In [2]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

courses_collection.delete_many({}) 

operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}),
    InsertOne({'course': 'Biology 101', 'enrollments': 28, 'department': 'Biology'}),
    InsertOne({'course': 'Chemistry 101', 'enrollments': 22, 'department': 'Chemistry'}),
    InsertOne({'course': 'Math 201', 'enrollments': 18, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 301', 'enrollments': 35, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 101', 'enrollments': 24, 'department': 'History'}),
    InsertOne({'course': 'Physics 101', 'enrollments': 30, 'department': 'Physics'}),
    InsertOne({'course': 'Biology 202', 'enrollments': 19, 'department': 'Biology'}),
    InsertOne({'course': 'Chemistry 202', 'enrollments': 29, 'department': 'Chemistry'}),
]

courses_collection.bulk_write(operations)
print('Courses inserted successfully.')


Courses inserted successfully.


In [17]:
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(course)

{'_id': ObjectId('690c70760ae66d5c9fb0dbd1'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690c70760ae66d5c9fb0dbd2'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}
{'_id': ObjectId('690c70760ae66d5c9fb0dbd5'), 'course': 'Biology 101', 'enrollments': 28, 'department': 'Biology'}
{'_id': ObjectId('690c70760ae66d5c9fb0dbd6'), 'course': 'Chemistry 101', 'enrollments': 22, 'department': 'Chemistry'}
{'_id': ObjectId('690c70760ae66d5c9fb0dbd8'), 'course': 'CS 301', 'enrollments': 35, 'department': 'Computer Science'}
{'_id': ObjectId('690c70760ae66d5c9fb0dbd9'), 'course': 'History 101', 'enrollments': 24, 'department': 'History'}
{'_id': ObjectId('690c70760ae66d5c9fb0dbda'), 'course': 'Physics 101', 'enrollments': 30, 'department': 'Physics'}
{'_id': ObjectId('690c70760ae66d5c9fb0dbdc'), 'course': 'Chemistry 202', 'enrollments': 29, 'department': 'Chemistry'}


In [18]:
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(course)

{'_id': ObjectId('690c70760ae66d5c9fb0dbd1'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690c70760ae66d5c9fb0dbd2'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}
{'_id': ObjectId('690c70760ae66d5c9fb0dbd7'), 'course': 'Math 201', 'enrollments': 18, 'department': 'Mathematics'}
{'_id': ObjectId('690c70760ae66d5c9fb0dbd8'), 'course': 'CS 301', 'enrollments': 35, 'department': 'Computer Science'}


In [19]:
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Physics', 'average_enrollment': 22.5}
{'_id': 'Computer Science', 'average_enrollment': 30.0}
{'_id': 'Mathematics', 'average_enrollment': 24.0}
{'_id': 'History', 'average_enrollment': 22.0}
{'_id': 'Chemistry', 'average_enrollment': 25.5}
{'_id': 'Biology', 'average_enrollment': 23.5}


In [20]:
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Physics', 'max_enrollment': 30}
{'_id': 'Computer Science', 'max_enrollment': 35}
{'_id': 'Mathematics', 'max_enrollment': 30}
{'_id': 'History', 'max_enrollment': 24}
{'_id': 'Chemistry', 'max_enrollment': 29}
{'_id': 'Biology', 'max_enrollment': 28}


In [21]:
pipeline = [
    {'$project': {
        'course_name': '$course', 'department_name': '$department', 'enrollments': 1, '_id': 0
     }}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'enrollments': 30, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'enrollments': 25, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'enrollments': 20, 'course_name': 'History 201', 'department_name': 'History'}
{'enrollments': 15, 'course_name': 'Physics 202', 'department_name': 'Physics'}
{'enrollments': 28, 'course_name': 'Biology 101', 'department_name': 'Biology'}
{'enrollments': 22, 'course_name': 'Chemistry 101', 'department_name': 'Chemistry'}
{'enrollments': 18, 'course_name': 'Math 201', 'department_name': 'Mathematics'}
{'enrollments': 35, 'course_name': 'CS 301', 'department_name': 'Computer Science'}
{'enrollments': 24, 'course_name': 'History 101', 'department_name': 'History'}
{'enrollments': 30, 'course_name': 'Physics 101', 'department_name': 'Physics'}
{'enrollments': 19, 'course_name': 'Biology 202', 'department_name': 'Biology'}
{'enrollments': 29, 'course_name': 'Chemistry 202', 'department_name': 'Chemistry'}


In [3]:
pipeline = [
    {'$addFields': {
        'enrollment_category': {
            '$cond': {
                'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'
            }
        }
     }}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': ObjectId('690c771308aff3ee56484acf'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'_id': ObjectId('690c771308aff3ee56484ad0'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'_id': ObjectId('690c771308aff3ee56484ad1'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': 'low'}
{'_id': ObjectId('690c771308aff3ee56484ad2'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrollment_category': 'low'}
{'_id': ObjectId('690c771308aff3ee56484ad3'), 'course': 'Biology 101', 'enrollments': 28, 'department': 'Biology', 'enrollment_category': 'high'}
{'_id': ObjectId('690c771308aff3ee56484ad4'), 'course': 'Chemistry 101', 'enrollments': 22, 'department': 'Chemistry', 'enrollment_category': 'high'}
{'_id': ObjectId('690c771308aff3ee56484ad5'), 'course': 'Math 201', 'enrollments': 18, 'department': 'Mathematics', '

In [5]:
courses_collection.delete_many({})

departments = ['Mathematics', 'Computer Science', 'History', 'Physics', 'Biology', 'Chemistry']
operations = [
    InsertOne({'course': 'Course 1_Mathematics', 'enrollments': 17, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 2_Computer', 'enrollments': 17, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 3_History', 'enrollments': 24, 'department': 'History'}),
    InsertOne({'course': 'Course 4_Physics', 'enrollments': 31, 'department': 'Physics'}),
    InsertOne({'course': 'Course 5_Biology', 'enrollments': 38, 'department': 'Biology'}),
    InsertOne({'course': 'Course 6_Chemistry', 'enrollments': 45, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 7_Mathematics', 'enrollments': 12, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 8_Computer', 'enrollments': 19, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 9_History', 'enrollments': 26, 'department': 'History'}),
    InsertOne({'course': 'Course 10_Physics', 'enrollments': 33, 'department': 'Physics'}),
    InsertOne({'course': 'Course 11_Biology', 'enrollments': 40, 'department': 'Biology'}),
    InsertOne({'course': 'Course 12_Chemistry', 'enrollments': 47, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 13_Mathematics', 'enrollments': 14, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 14_Computer', 'enrollments': 21, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 15_History', 'enrollments': 28, 'department': 'History'}),
    InsertOne({'course': 'Course 16_Physics', 'enrollments': 35, 'department': 'Physics'}),
    InsertOne({'course': 'Course 17_Biology', 'enrollments': 42, 'department': 'Biology'}),
    InsertOne({'course': 'Course 18_Chemistry', 'enrollments': 49, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 19_Mathematics', 'enrollments': 16, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 20_Computer', 'enrollments': 23, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 21_History', 'enrollments': 30, 'department': 'History'}),
    InsertOne({'course': 'Course 22_Physics', 'enrollments': 37, 'department': 'Physics'}),
    InsertOne({'course': 'Course 23_Biology', 'enrollments': 44, 'department': 'Biology'}),
    InsertOne({'course': 'Course 24_Chemistry', 'enrollments': 11, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 25_Mathematics', 'enrollments': 18, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 26_Computer', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 27_History', 'enrollments': 32, 'department': 'History'}),
    InsertOne({'course': 'Course 28_Physics', 'enrollments': 39, 'department': 'Physics'}),
    InsertOne({'course': 'Course 29_Biology', 'enrollments': 46, 'department': 'Biology'}),
    InsertOne({'course': 'Course 30_Chemistry', 'enrollments': 13, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 31_Mathematics', 'enrollments': 20, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 32_Computer', 'enrollments': 27, 'department': 'Computer Science'}), # CS > 25
    InsertOne({'course': 'Course 33_History', 'enrollments': 34, 'department': 'History'}),
    InsertOne({'course': 'Course 34_Physics', 'enrollments': 41, 'department': 'Physics'}),
    InsertOne({'course': 'Course 35_Biology', 'enrollments': 48, 'department': 'Biology'}),
    InsertOne({'course': 'Course 36_Chemistry', 'enrollments': 15, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 37_Mathematics', 'enrollments': 22, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 38_Computer', 'enrollments': 29, 'department': 'Computer Science'}), # CS > 25
    InsertOne({'course': 'Course 39_History', 'enrollments': 36, 'department': 'History'}),
    InsertOne({'course': 'Course 40_Physics', 'enrollments': 43, 'department': 'Physics'}),
    InsertOne({'course': 'Course 41_Biology', 'enrollments': 10, 'department': 'Biology'}),
    InsertOne({'course': 'Course 42_Chemistry', 'enrollments': 17, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 43_Mathematics', 'enrollments': 24, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 44_Computer', 'enrollments': 31, 'department': 'Computer Science'}), # CS > 25
    InsertOne({'course': 'Course 45_History', 'enrollments': 38, 'department': 'History'}),
    InsertOne({'course': 'Course 46_Physics', 'enrollments': 45, 'department': 'Physics'}),
    InsertOne({'course': 'Course 47_Biology', 'enrollments': 12, 'department': 'Biology'}),
    InsertOne({'course': 'Course 48_Chemistry', 'enrollments': 19, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 49_Mathematics', 'enrollments': 26, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 50_Computer', 'enrollments': 33, 'department': 'Computer Science'}), # CS > 25
    InsertOne({'course': 'Course 51_History', 'enrollments': 40, 'department': 'History'}),
    InsertOne({'course': 'Course 52_Physics', 'enrollments': 47, 'department': 'Physics'}),
    InsertOne({'course': 'Course 53_Biology', 'enrollments': 14, 'department': 'Biology'}),
    InsertOne({'course': 'Course 54_Chemistry', 'enrollments': 21, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 55_Mathematics', 'enrollments': 28, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 56_Computer', 'enrollments': 35, 'department': 'Computer Science'}), # CS > 25
    InsertOne({'course': 'Course 57_History', 'enrollments': 42, 'department': 'History'}),
    InsertOne({'course': 'Course 58_Physics', 'enrollments': 49, 'department': 'Physics'}),
    InsertOne({'course': 'Course 59_Biology', 'enrollments': 16, 'department': 'Biology'}),
    InsertOne({'course': 'Course 60_Chemistry', 'enrollments': 23, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 61_Mathematics', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 62_Computer', 'enrollments': 37, 'department': 'Computer Science'}), # CS > 25
    InsertOne({'course': 'Course 63_History', 'enrollments': 44, 'department': 'History'}),
    InsertOne({'course': 'Course 64_Physics', 'enrollments': 11, 'department': 'Physics'}),
    InsertOne({'course': 'Course 65_Biology', 'enrollments': 18, 'department': 'Biology'}),
    InsertOne({'course': 'Course 66_Chemistry', 'enrollments': 25, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 67_Mathematics', 'enrollments': 32, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 68_Computer', 'enrollments': 39, 'department': 'Computer Science'}), # CS > 25
    InsertOne({'course': 'Course 69_History', 'enrollments': 46, 'department': 'History'}),
    InsertOne({'course': 'Course 70_Physics', 'enrollments': 13, 'department': 'Physics'}),
    InsertOne({'course': 'Course 71_Biology', 'enrollments': 20, 'department': 'Biology'}),
    InsertOne({'course': 'Course 72_Chemistry', 'enrollments': 27, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 73_Mathematics', 'enrollments': 34, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 74_Computer', 'enrollments': 41, 'department': 'Computer Science'}), # CS > 25
    InsertOne({'course': 'Course 75_History', 'enrollments': 48, 'department': 'History'}),
    InsertOne({'course': 'Course 76_Physics', 'enrollments': 15, 'department': 'Physics'}),
    InsertOne({'course': 'Course 77_Biology', 'enrollments': 22, 'department': 'Biology'}),
    InsertOne({'course': 'Course 78_Chemistry', 'enrollments': 29, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 79_Mathematics', 'enrollments': 36, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 80_Computer', 'enrollments': 43, 'department': 'Computer Science'}), # CS > 25
    InsertOne({'course': 'Course 81_History', 'enrollments': 10, 'department': 'History'}),
    InsertOne({'course': 'Course 82_Physics', 'enrollments': 17, 'department': 'Physics'}),
    InsertOne({'course': 'Course 83_Biology', 'enrollments': 24, 'department': 'Biology'}),
    InsertOne({'course': 'Course 84_Chemistry', 'enrollments': 31, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 85_Mathematics', 'enrollments': 38, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 86_Computer', 'enrollments': 45, 'department': 'Computer Science'}), # CS > 25
    InsertOne({'course': 'Course 87_History', 'enrollments': 12, 'department': 'History'}),
    InsertOne({'course': 'Course 88_Physics', 'enrollments': 19, 'department': 'Physics'}),
    InsertOne({'course': 'Course 89_Biology', 'enrollments': 26, 'department': 'Biology'}),
    InsertOne({'course': 'Course 90_Chemistry', 'enrollments': 33, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 91_Mathematics', 'enrollments': 40, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 92_Computer', 'enrollments': 47, 'department': 'Computer Science'}), # CS > 25
    InsertOne({'course': 'Course 93_History', 'enrollments': 14, 'department': 'History'}),
    InsertOne({'course': 'Course 94_Physics', 'enrollments': 21, 'department': 'Physics'}),
    InsertOne({'course': 'Course 95_Biology', 'enrollments': 28, 'department': 'Biology'}),
    InsertOne({'course': 'Course 96_Chemistry', 'enrollments': 35, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 97_Mathematics', 'enrollments': 42, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 98_Computer', 'enrollments': 49, 'department': 'Computer Science'}), # CS > 25
    InsertOne({'course': 'Course 99_History', 'enrollments': 16, 'department': 'History'}),
    InsertOne({'course': 'Course 100_Physics', 'enrollments': 23, 'department': 'Physics'}),
    InsertOne({'course': 'Course 101_Biology', 'enrollments': 30, 'department': 'Biology'}),
    InsertOne({'course': 'Course 102_Chemistry', 'enrollments': 37, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 103_Mathematics', 'enrollments': 44, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 104_Computer', 'enrollments': 11, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 105_History', 'enrollments': 18, 'department': 'History'}),
    InsertOne({'course': 'Course 106_Physics', 'enrollments': 25, 'department': 'Physics'}),
    InsertOne({'course': 'Course 107_Biology', 'enrollments': 32, 'department': 'Biology'}),
    InsertOne({'course': 'Course 108_Chemistry', 'enrollments': 39, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 109_Mathematics', 'enrollments': 46, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 110_Computer', 'enrollments': 13, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 111_History', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Course 112_Physics', 'enrollments': 27, 'department': 'Physics'}),
    InsertOne({'course': 'Course 113_Biology', 'enrollments': 34, 'department': 'Biology'}),
    InsertOne({'course': 'Course 114_Chemistry', 'enrollments': 41, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 115_Mathematics', 'enrollments': 48, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 116_Computer', 'enrollments': 15, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 117_History', 'enrollments': 22, 'department': 'History'}),
    InsertOne({'course': 'Course 118_Physics', 'enrollments': 29, 'department': 'Physics'}),
    InsertOne({'course': 'Course 119_Biology', 'enrollments': 36, 'department': 'Biology'}),
    InsertOne({'course': 'Course 120_Chemistry', 'enrollments': 43, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 121_Mathematics', 'enrollments': 10, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 122_Computer', 'enrollments': 17, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 123_History', 'enrollments': 24, 'department': 'History'}),
    InsertOne({'course': 'Course 124_Physics', 'enrollments': 31, 'department': 'Physics'}),
    InsertOne({'course': 'Course 125_Biology', 'enrollments': 38, 'department': 'Biology'}),
    InsertOne({'course': 'Course 126_Chemistry', 'enrollments': 45, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 127_Mathematics', 'enrollments': 12, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 128_Computer', 'enrollments': 19, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 129_History', 'enrollments': 26, 'department': 'History'}),
    InsertOne({'course': 'Course 130_Physics', 'enrollments': 33, 'department': 'Physics'}),
    InsertOne({'course': 'Course 131_Biology', 'enrollments': 40, 'department': 'Biology'}),
    InsertOne({'course': 'Course 132_Chemistry', 'enrollments': 47, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 133_Mathematics', 'enrollments': 14, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 134_Computer', 'enrollments': 21, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 135_History', 'enrollments': 28, 'department': 'History'}),
    InsertOne({'course': 'Course 136_Physics', 'enrollments': 35, 'department': 'Physics'}),
    InsertOne({'course': 'Course 137_Biology', 'enrollments': 42, 'department': 'Biology'}),
    InsertOne({'course': 'Course 138_Chemistry', 'enrollments': 49, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 139_Mathematics', 'enrollments': 16, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 140_Computer', 'enrollments': 23, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 141_History', 'enrollments': 30, 'department': 'History'}),
    InsertOne({'course': 'Course 142_Physics', 'enrollments': 37, 'department': 'Physics'}),
    InsertOne({'course': 'Course 143_Biology', 'enrollments': 44, 'department': 'Biology'}),
    InsertOne({'course': 'Course 144_Chemistry', 'enrollments': 11, 'department': 'Chemistry'}),
    InsertOne({'course': 'Course 145_Mathematics', 'enrollments': 18, 'department': 'Mathematics'}),
    InsertOne({'course': 'Course 146_Computer', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'Course 147_History', 'enrollments': 32, 'department': 'History'}),
    InsertOne({'course': 'Course 148_Physics', 'enrollments': 39, 'department': 'Physics'}),
    InsertOne({'course': 'Course 149_Biology', 'enrollments': 46, 'department': 'Biology'}),
    InsertOne({'course': 'Course 150_Chemistry', 'enrollments': 13, 'department': 'Chemistry'}),
]

# Bulk insert 150 data
courses_collection.bulk_write(operations)
print('Courses inserted successfully.')

Courses inserted successfully.


In [6]:
count_pipeline = [
    {'$group': {
        '_id': '$department', 'count_courses': {'$sum': 1}
    }},
    {'$sort': {'count_courses': -1}}
]
for result in courses_collection.aggregate(count_pipeline):
    print(result)

{'_id': 'Physics', 'count_courses': 25}
{'_id': 'Computer Science', 'count_courses': 25}
{'_id': 'Mathematics', 'count_courses': 25}
{'_id': 'History', 'count_courses': 25}
{'_id': 'Biology', 'count_courses': 25}
{'_id': 'Chemistry', 'count_courses': 25}


In [9]:
match_group_pipeline = [
    {'$match': {
        'department': 'Computer Science', 'enrollments': {'$gt': 25}
    }},

    {'$group': {
        '_id': '$department', 'total_enrollments_matched': {'$sum': '$enrollments'}, 'count_matched_courses': {'$sum': 1} 
    }}
]
results = list(courses_collection.aggregate(match_group_pipeline))

if not results:
    print("Tidak ada kursus 'Computer Science' dengan pendaftaran > 25.")
else:
    for result in results:
        print(result)

{'_id': 'Computer Science', 'total_enrollments_matched': 456, 'count_matched_courses': 12}


In [8]:
students_collection = db['students']
students_collection.delete_many({}) 

students_data = [
    {'name': 'Alice', 'major': 'Computer Science', 'course_name': 'Course 2_Computer'},
    {'name': 'Bob', 'major': 'Mathematics', 'course_name': 'Course 1_Mathematics'},
    {'name': 'Charlie', 'major': 'History', 'course_name': 'Course 3_History'},
    {'name': 'David', 'major': 'Mathematics', 'course_name': 'Course 1_Mathematics'},
    {'name': 'Eve', 'major': 'Computer Science', 'course_name': 'Course 8_Computer'}
]
students_collection.insert_many(students_data)
print(f"\n{students_collection.count_documents({})} students inserted.")


print("\n--- Menggabungkan (JOIN) Kursus dengan Siswa ($lookup) ---")
lookup_pipeline = [
    {'$lookup': {
        'from': 'students', 'localField': 'course', 'foreignField': 'course_name', 'as': 'student_list'
    }},
    {'$match': {'student_list': {'$ne': []}}}, {'$limit': 5}
]
for result in courses_collection.aggregate(lookup_pipeline):
    print(result)


5 students inserted.

--- Menggabungkan (JOIN) Kursus dengan Siswa ($lookup) ---
{'_id': ObjectId('6913b08eebe0f89ffe58f857'), 'course': 'Course 1_Mathematics', 'enrollments': 17, 'department': 'Mathematics', 'student_list': [{'_id': ObjectId('6913b1afebe0f89ffe58f8ee'), 'name': 'Bob', 'major': 'Mathematics', 'course_name': 'Course 1_Mathematics'}, {'_id': ObjectId('6913b1afebe0f89ffe58f8f0'), 'name': 'David', 'major': 'Mathematics', 'course_name': 'Course 1_Mathematics'}]}
{'_id': ObjectId('6913b08eebe0f89ffe58f858'), 'course': 'Course 2_Computer', 'enrollments': 17, 'department': 'Computer Science', 'student_list': [{'_id': ObjectId('6913b1afebe0f89ffe58f8ed'), 'name': 'Alice', 'major': 'Computer Science', 'course_name': 'Course 2_Computer'}]}
{'_id': ObjectId('6913b08eebe0f89ffe58f859'), 'course': 'Course 3_History', 'enrollments': 24, 'department': 'History', 'student_list': [{'_id': ObjectId('6913b1afebe0f89ffe58f8ef'), 'name': 'Charlie', 'major': 'History', 'course_name': 'Cours